In [1]:
import numpy as np
from tqdm.notebook import tqdm

In [2]:
global register_a
global register_b
global register_c

global ip

def read_combo(operand):
    global register_a
    global register_b
    global register_c
    if 0 <= operand <= 3: return operand
    if operand == 4: return register_a
    if operand == 5: return register_b
    if operand == 6: return register_c
    if operand == 7: raise ValueError("opcode 7 reserver")
    raise ValueError("opcode invalid")

def adv(operand):
    global register_a
    register_a = int(register_a / (2**read_combo(operand)))
    return ""

def bxl(operand):
    global register_b
    register_b = register_b ^ operand
    return ""
    
def bst(operand):
    global register_b
    register_b = read_combo(operand) % 8
    return ""

def jnz(operand):
    global ip
    if register_a != 0: ip = operand - 2
    return ""
    
def bxc(operand):
    global register_b
    register_b = register_b ^ register_c
    return ""
    
def out(operand):
    return str(read_combo(operand) % 8) + ","
    
def bdv(operand):
    global register_b
    register_b = int(register_a / (2**read_combo(operand)))
    return ""
    
def cdv(operand):
    global register_c
    register_c = int(register_a / (2**read_combo(operand)))
    return ""


opcode = {
    0: adv,
    1: bxl,
    2: bst,
    3: jnz,
    4: bxc,
    5: out,
    6: bdv,
    7: cdv
          }

In [3]:
def run_program(program, reg_a, reg_b, reg_c):
    global register_a
    global register_b
    global register_c
    global ip
    
    register_a = reg_a
    register_b = reg_b
    register_c = reg_c
    
    ip = 0
    
    std_out = ""
    
    repeated_state = {}
    
    while(0<= ip < len(program) - 1):
        state = (ip, register_a, register_b, register_c)
        if repeated_state.get(state, 0): break
        repeated_state[state] = 1

        std_out += opcode[program[ip]](program[ip + 1])
        ip += 2
        
        if len(std_out) > 30: break
        
        
    return std_out[:-1]


In [26]:
with open("d17_input") as f:
    reg_a = f.readline()[:-1]
    reg_a = int(reg_a.split(":")[-1])
    
    reg_b = f.readline()[:-1]
    reg_b = int(reg_b.split(":")[-1])
    
    reg_c = f.readline()[:-1]
    reg_c = int(reg_c.split(":")[1])
    
    f.readline()
    program = f.readline()[:-1]
    program = np.array(program.split(":")[-1].split(","), dtype=int)

In [28]:
run_program(program, reg_a, reg_b, reg_c)

'7,5,4,3,4,5,3,4,6'

In [18]:
output = {}
possible_inputs = [0]
target = 4


for instr in tqdm(np.flip(program)):
    lo = int(target - min(100000, target/2))
    hi = int(target + min(100000, target/2))

    for i in (range(lo,hi)):
        std_out = run_program(program[:-2], i, 0, 0)
        
        output[std_out] =  output.get(std_out, {})
        output[std_out][register_a] = output[std_out].get(register_a, [])
        output[std_out][register_a].append(i)
        
    new_possible_inputs = []
    for p in possible_inputs:
        new_possible_inputs = np.hstack((
            new_possible_inputs,
            (output[str(instr)].get(p, []))
        ))
    possible_inputs = new_possible_inputs
    print(f"{instr}\t{int(possible_inputs[0])}")
    target = int(possible_inputs[0]) * 8
    
    

  0%|          | 0/16 [00:00<?, ?it/s]

0	4
3	37
3	298
0	2390
5	19124
5	152996
3	1223973
4	9791785
5	78334283
1	626674267
5	5013394138
7	40107153110
1	320857224887
1	2566857799098
4	20534862392784
2	164278899142333
